## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-19-23-00-15 +0000,nyt,Live Updates: Former Prince Andrew Is Released...,https://www.nytimes.com/live/2026/02/19/world/...
1,2026-02-19-23-00-14 +0000,nyt,"British Police Arrest Former Prince Andrew, in...",https://www.nytimes.com/2026/02/19/world/europ...
2,2026-02-19-23-00-07 +0000,nyt,Former Prince Andrew’s Arrest Casts Shadow Ove...,https://www.nytimes.com/live/2026/02/19/world/...
3,2026-02-19-22-59-54 +0000,nypost,UN nuclear watchdog chief urges US to secure I...,https://nypost.com/2026/02/19/world-news/un-nu...
4,2026-02-19-22-59-52 +0000,nyt,"Alysa Liu Is Skating Again, Her Way This Time",https://www.nytimes.com/2026/02/19/world/olymp...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2474/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
57,trump,45
4,andrew,31
28,iran,27
3,prince,27
9,epstein,26


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
111,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...,169
100,2026-02-19-20-28-03 +0000,nyt,The police are investigating former Prince And...,https://www.nytimes.com/live/2026/02/19/world/...,151
125,2026-02-19-19-31-44 +0000,nypost,Ex-Prince Andrew looks aghast in first photos ...,https://nypost.com/2026/02/19/world-news/ex-pr...,142
0,2026-02-19-23-00-15 +0000,nyt,Live Updates: Former Prince Andrew Is Released...,https://www.nytimes.com/live/2026/02/19/world/...,141
1,2026-02-19-23-00-14 +0000,nyt,"British Police Arrest Former Prince Andrew, in...",https://www.nytimes.com/2026/02/19/world/europ...,141


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
111,169,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...
320,97,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...
263,71,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...
218,71,2026-02-19-14-39-21 +0000,nypost,White House warns Iran it’d be ‘very wise’ to ...,https://nypost.com/2026/02/19/us-news/white-ho...
303,61,2026-02-19-09-58-53 +0000,nypost,Rep. Lauren Boebert’s son Tyler slapped with a...,https://nypost.com/2026/02/19/us-news/lauren-b...
255,53,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...
331,49,2026-02-19-05-01-09 +0000,nyt,Policy Flip-Flops Hurt the British Leader. The...,https://www.nytimes.com/2026/02/19/world/europ...
203,47,2026-02-19-15-39-00 +0000,wsj,"The U.S. trade deficit jumped in December, the...",https://www.wsj.com/economy/trade/u-s-trade-de...
351,44,2026-02-19-02-27-00 +0000,wsj,The top Republican challenger to Rep. Tony Gon...,https://www.wsj.com/politics/elections/texas-t...
93,41,2026-02-19-20-36-27 +0000,nypost,Two Lake Tahoe avalanche victims ID’d as siste...,https://nypost.com/2026/02/19/us-news/idaho-si...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
